In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

In [2]:
compas = pd.read_csv('data/cox-violent-parsed.csv')
compas.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event
0,1.0,miguel hernandez,miguel,hernandez,14/08/2013,Male,18/04/1947,69,Greater than 45,Other,...,Risk of Violence,1,Low,14/08/2013,07/07/2014,14/07/2014,0,0,327,0
1,2.0,miguel hernandez,miguel,hernandez,14/08/2013,Male,18/04/1947,69,Greater than 45,Other,...,Risk of Violence,1,Low,14/08/2013,07/07/2014,14/07/2014,0,334,961,0
2,3.0,michael ryan,michael,ryan,31/12/2014,Male,06/02/1985,31,25 - 45,Caucasian,...,Risk of Violence,2,Low,31/12/2014,30/12/2014,03/01/2015,0,3,457,0
3,4.0,kevon dixon,kevon,dixon,27/01/2013,Male,22/01/1982,34,25 - 45,African-American,...,Risk of Violence,1,Low,27/01/2013,26/01/2013,05/02/2013,0,9,159,1
4,5.0,ed philo,ed,philo,14/04/2013,Male,14/05/1991,24,Less than 25,African-American,...,Risk of Violence,3,Low,14/04/2013,16/06/2013,16/06/2013,4,0,63,0


In [3]:
compas['c_jail_in'] = pd.to_datetime(compas['c_jail_in'], format='mixed')
compas['c_jail_out'] = pd.to_datetime(compas['c_jail_out'], format='mixed')

compas['v_screening_date'] = pd.to_datetime(compas['v_screening_date'], format='mixed')
compas['vr_offense_date'] = pd.to_datetime(compas['vr_offense_date'], format='mixed')

compas['c_offense_date'] = pd.to_datetime(compas['c_offense_date'], format='mixed')
compas['c_arrest_date'] = pd.to_datetime(compas['c_arrest_date'], format='mixed')

compas['compas_screening_date'] = pd.to_datetime(compas['compas_screening_date'], format='mixed')

ValueError: time data '13/08/2013 6:03' does not match format 'mixed' (match)

In [ ]:
compas = compas.drop(['first','last','c_case_number','c_charge_degree', 'c_charge_desc', 'r_case_number','r_charge_degree','vr_case_number','vr_charge_degree','start','end','event'],axis=1)

#remove row with nan scores 
compas = compas.loc[compas['score_text'].notnull()]
#clean out the "-1" decile score from dataset 
compas = compas.loc[compas['v_decile_score'] != -1]

#label encodimg
compas['sex'] = compas['sex'].replace({'Male': 1, 'Female': 0})
compas['score_text'] = compas['score_text'].replace({'Low':0, 'Medium':1, 'High': 2})

#days in jail
compas['jail_out- jail_in'] = (compas['c_jail_out'] - compas['c_jail_in']).dt.days

In [ ]:
compas.info()

In [ ]:
compas.describe(include=["object","category"]).T

In [ ]:
compas.describe().T

In [ ]:
compas_race = compas['race'].value_counts()
compas_race

In [ ]:
compas['race'].value_counts().plot(kind='bar')
plt.xticks(rotation=45)

In [ ]:
compas['age_cat'].value_counts().plot(kind='bar')
plt.xticks(rotation=45)

In [ ]:
compas.info()

In [ ]:
corr_matrix = compas[['sex', 'age','juv_fel_count', 'decile_score', 
                    'juv_misd_count','juv_other_count','priors_count', 
                    'days_b_screening_arrest', 'c_days_from_compas', 'r_days_from_arrest', 'v_decile_score', 'jail_out- jail_in','score_text' ]].corr()
sns.heatmap(corr_matrix, annot=False)
plt.show()

In [ ]:
corr_matrix.style.background_gradient(cmap='coolwarm')

In [ ]:
compas.head()

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


compas['reoffend'] = compas['v_decile_score'].apply(lambda x: 1 if x > 5 else 0)

# Selecting features and target
features = compas[['age', 'priors_count']]
target = compas['reoffend']

# Splitting the data into training and testing sets
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.2, random_state=42)

# Initializing and training the logistic regression model
model = LogisticRegression()
model.fit(train_features, train_target)

# Making predictions
predicted_classes = model.predict(test_features)

# Evaluating the model
accuracy = accuracy_score(test_target, predicted_classes)
report = classification_report(test_target, predicted_classes)
conf_matrix = confusion_matrix(test_target, predicted_classes)

report

'              precision    recall  f1-score   support\n\n           0       0.85      0.90      0.87      2631\n           1       0.70      0.60      0.64      1033\n\n    accuracy                           0.81      3664\n   macro avg       0.77      0.75      0.76      3664\nweighted avg       0.81      0.81      0.81      3664\n'

In [8]:
# Reformatting the classification report for better readability
from tabulate import tabulate

# Converting the classification report into a dictionary
report_dict = classification_report(test_target, predicted_classes, output_dict=True)

# Preparing data for tabulation
report_data = []
for key, value in report_dict.items():
    if key == 'accuracy':
        report_data.append(['accuracy', '', '', value, report_dict['macro avg']['support']])
    elif key in ['macro avg', 'weighted avg']:
        report_data.append([key, value['precision'], value['recall'], value['f1-score'], value['support']])
    else:
        report_data.append([f'Class {key}', value['precision'], value['recall'], value['f1-score'], value['support']])

# Creating a table with headers
headers = ["Metric", "Precision", "Recall", "F1-Score", "Support"]
table = tabulate(report_data, headers, tablefmt="pretty")

# Printing the formatted table
print(table)



+--------------+--------------------+--------------------+--------------------+---------+
|    Metric    |     Precision      |       Recall       |      F1-Score      | Support |
+--------------+--------------------+--------------------+--------------------+---------+
|   Class 0    | 0.8513172140021653 | 0.8966172557962752 | 0.873380229544613  | 2631.0  |
|   Class 1    | 0.6954087346024636 | 0.601161665053243  | 0.6448598130841121 | 1033.0  |
|   accuracy   |                    |                    | 0.8133187772925764 | 3664.0  |
|  macro avg   | 0.7733629743023145 | 0.7488894604247591 | 0.7591200213143625 | 3664.0  |
| weighted avg | 0.8073615755687887 | 0.8133187772925764 | 0.8089529396418571 | 3664.0  |
+--------------+--------------------+--------------------+--------------------+---------+


In [9]:
# Example data
new_data = pd.DataFrame({
    'age': [25, 40],
    'priors_count': [10, 0]
})

# Making predictions
predictions = model.predict(new_data)

# Interpreting predictions
prediction_labels = ['Likely to reoffend' if pred == 1 else 'Less likely to reoffend' for pred in predictions]

# Display results
for i, label in enumerate(prediction_labels):
    print(f"Individual {i+1}: {label}")


Individual 1: Likely to reoffend
Individual 2: Less likely to reoffend


In [11]:
# Assuming features and target have been prepared

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the neural network architecture
model = Sequential()
model.add(Dense(10, input_dim=2, activation='relu'))  # Input layer and first hidden layer
model.add(Dense(1, activation='sigmoid'))  # Output layer

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_features, train_target, epochs=100, batch_size=10, validation_split=0.1)

# Evaluate the model
accuracy = model.evaluate(test_features, test_target)

# Make predictions
predictions = model.predict(new_data)


Epoch 1/100
1319/1319 [==============================] - 1s 319us/step - loss: 0.5203 - accuracy: 0.7193 - val_loss: 0.5021 - val_accuracy: 0.7360
Epoch 2/100
1319/1319 [==============================] - 0s 272us/step - loss: 0.4813 - accuracy: 0.7384 - val_loss: 0.4840 - val_accuracy: 0.7381
Epoch 3/100
1319/1319 [==============================] - 0s 271us/step - loss: 0.4726 - accuracy: 0.7562 - val_loss: 0.4833 - val_accuracy: 0.7483
Epoch 4/100
1319/1319 [==============================] - 0s 270us/step - loss: 0.4710 - accuracy: 0.7698 - val_loss: 0.4796 - val_accuracy: 0.7804
Epoch 5/100
1319/1319 [==============================] - 0s 271us/step - loss: 0.4692 - accuracy: 0.7789 - val_loss: 0.4782 - val_accuracy: 0.8008
Epoch 6/100
1319/1319 [==============================] - 0s 272us/step - loss: 0.4678 - accuracy: 0.7842 - val_loss: 0.4841 - val_accuracy: 0.7640
Epoch 7/100
1319/1319 [==============================] - 0s 272us/step - loss: 0.4677 - accuracy: 0.7870 - val_loss: 0

1319/1319 [==============================] - 0s 275us/step - loss: 0.4254 - accuracy: 0.8040 - val_loss: 0.4435 - val_accuracy: 0.8042
Epoch 57/100
1319/1319 [==============================] - 0s 276us/step - loss: 0.4265 - accuracy: 0.8045 - val_loss: 0.4460 - val_accuracy: 0.8042
Epoch 58/100
1319/1319 [==============================] - 0s 275us/step - loss: 0.4256 - accuracy: 0.8057 - val_loss: 0.4454 - val_accuracy: 0.8022
Epoch 59/100
1319/1319 [==============================] - 0s 275us/step - loss: 0.4253 - accuracy: 0.8033 - val_loss: 0.4469 - val_accuracy: 0.8070
Epoch 60/100
1319/1319 [==============================] - 0s 274us/step - loss: 0.4273 - accuracy: 0.8030 - val_loss: 0.4412 - val_accuracy: 0.7995
Epoch 61/100
1319/1319 [==============================] - 0s 274us/step - loss: 0.4256 - accuracy: 0.8028 - val_loss: 0.4370 - val_accuracy: 0.8104
Epoch 62/100
1319/1319 [==============================] - 0s 279us/step - loss: 0.4255 - accuracy: 0.8029 - val_loss: 0.4414 

In [13]:
# Make predictions
predictions = model.predict(new_data)

predictions


1/1 [==============================] - 0s 12ms/step


array([[0.5547271 ],
       [0.02580232]], dtype=float32)